In [1]:
import mitiq

mitiq.SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'cirq', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [2]:
frontend = "cirq"

In [3]:
from mitiq import benchmarks

circuit = benchmarks.generate_rb_circuits(
  n_qubits=1, num_cliffords=2, return_type = frontend,
)[0]

print(circuit)

0: ───X^0.5───Y^0.5───Y^-0.5───I───X^-0.5───


In [4]:
import numpy as np
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

In [7]:
from mitiq.zne import scaled_circuits, combine_results
from mitiq.zne.scaling import fold_gates_at_random

scale_factors = 2.0

folded_circuits = scaled_circuits(
    circuit=circuit,
    scale_factors=[scale_factors],
    scale_method=fold_gates_at_random,
)

folded_circuits

[0: ───X^0.5───X^-0.5───X^0.5───Y^0.5───Y^-0.5───I───X^-0.5───X^0.5───X^-0.5───]

In [8]:
results = [execute(circuit) for circuit in folded_circuits]

print(results)

[0.9431023]
